In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
text = open("shakespeare.txt","r").read()

In [4]:
text[:500]

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bu"

In [5]:
type(text)

str

In [6]:
vocab = sorted(set(text))

In [9]:
char_to_ind = {u:i for i,u in enumerate(vocab)}
print(char_to_ind)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, ',': 8, '-': 9, '.': 10, '0': 11, '1': 12, '2': 13, '3': 14, '4': 15, '5': 16, '6': 17, '7': 18, '8': 19, '9': 20, ':': 21, ';': 22, '<': 23, '>': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'I': 34, 'J': 35, 'K': 36, 'L': 37, 'M': 38, 'N': 39, 'O': 40, 'P': 41, 'Q': 42, 'R': 43, 'S': 44, 'T': 45, 'U': 46, 'V': 47, 'W': 48, 'X': 49, 'Y': 50, 'Z': 51, '[': 52, ']': 53, '_': 54, '`': 55, 'a': 56, 'b': 57, 'c': 58, 'd': 59, 'e': 60, 'f': 61, 'g': 62, 'h': 63, 'i': 64, 'j': 65, 'k': 66, 'l': 67, 'm': 68, 'n': 69, 'o': 70, 'p': 71, 'q': 72, 'r': 73, 's': 74, 't': 75, 'u': 76, 'v': 77, 'w': 78, 'x': 79, 'y': 80, 'z': 81, '|': 82, '}': 83}


In [10]:
ind_to_char = np.array(vocab)

In [11]:
print(ind_to_char)

['\n' ' ' '!' '"' '&' "'" '(' ')' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6'
 '7' '8' '9' ':' ';' '<' '>' '?' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J'
 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' '[' ']'
 '_' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p'
 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '|' '}']


In [14]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [15]:
encoded_text[:100]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 73, 70, 68,  1, 61, 56, 64,
       73, 60, 74, 75,  1, 58, 73, 60, 56, 75, 76, 73, 60, 74,  1, 78, 60,
        1, 59, 60, 74, 64, 73, 60,  1, 64, 69, 58, 73, 60, 56, 74, 60,  8,
        0,  1,  1, 45, 63, 56, 75,  1, 75, 63, 60, 73, 60, 57, 80,  1, 57,
       60, 56, 76, 75, 80,  5, 74,  1, 73, 70, 74, 60,  1, 68, 64])

In [16]:
import tensorflow as tf

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [17]:
seq_len = 120

In [18]:
total_seq = len(text)//seq_len

In [19]:
char_dataset =  tf.data.Dataset.from_tensor_slices(encoded_text)

In [22]:
for i in char_dataset.take(500):
    print(ind_to_char[i.numpy()],end="")


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu

The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [23]:
sequences = char_dataset.batch(seq_len+1,drop_remainder=True)

In [24]:
type(sequences)

tensorflow.python.data.ops.dataset_ops.BatchDataset

In [28]:
for i in sequences.take(1):
    print(i)
    

tf.Tensor(
[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75
  1], shape=(121,), dtype=int32)


In [29]:
def create_seq_target(seq):
    inp = seq[:-1]
    oup = seq[1:]
    return inp,oup

In [31]:
dataset = sequences.map(create_seq_target)

### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [32]:
batch_size = 128

buf_size=10000

dataset = dataset.shuffle(buf_size).batch(batch_size=batch_size,drop_remainder=True)

In [55]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

### Creating the Model

In [56]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,GRU,Dropout,Embedding

In [35]:
##### Setting up the loss function

In [57]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [58]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [59]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [60]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (128, None, 64)           5376      
_________________________________________________________________
gru_4 (GRU)                  (128, None, 1026)         3361176   
_________________________________________________________________
dense_4 (Dense)              (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [61]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [62]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices

<tf.Tensor: id=12247, shape=(120, 1), dtype=int64, numpy=
array([[61],
       [ 5],
       [57],
       [80],
       [20],
       [34],
       [40],
       [64],
       [23],
       [58],
       [46],
       [ 0],
       [56],
       [14],
       [18],
       [30],
       [79],
       [59],
       [10],
       [31],
       [50],
       [79],
       [24],
       [49],
       [69],
       [54],
       [13],
       [54],
       [75],
       [23],
       [76],
       [50],
       [ 9],
       [ 4],
       [35],
       [72],
       [44],
       [24],
       [79],
       [61],
       [58],
       [58],
       [23],
       [42],
       [62],
       [60],
       [79],
       [51],
       [66],
       [21],
       [47],
       [ 8],
       [74],
       [50],
       [38],
       [41],
       [ 4],
       [ 1],
       [11],
       [34],
       [38],
       [50],
       [78],
       [10],
       [15],
       [67],
       [54],
       [77],
       [23],
       [39],
       [60],
       [30],
      

In [63]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([61,  5, 57, 80, 20, 34, 40, 64, 23, 58, 46,  0, 56, 14, 18, 30, 79,
       59, 10, 31, 50, 79, 24, 49, 69, 54, 13, 54, 75, 23, 76, 50,  9,  4,
       35, 72, 44, 24, 79, 61, 58, 58, 23, 42, 62, 60, 79, 51, 66, 21, 47,
        8, 74, 50, 38, 41,  4,  1, 11, 34, 38, 50, 78, 10, 15, 67, 54, 77,
       23, 39, 60, 30, 42, 48, 69, 59, 10, 77, 55, 65, 71, 54, 79, 82, 15,
       62, 23, 79,  3, 26, 18, 39, 20, 59,  2, 38, 72, 83,  5,  2,  6, 22,
       25, 82, 24, 23, 37, 37, 34, 37,  5, 21, 82, 26, 56,  7, 75, 38, 77,
       82], dtype=int64)

In [41]:
from tensorflow.keras.models import load_model

In [64]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [65]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_5 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [66]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
    '''
      model: Trained Model to Generate Text
    start_seed: Intial Seed text in string form
    gen_size: Number of characters to generate

    Basic idea behind this function is to take in some seed text, format it so
    that it is in the correct shape for our network, then loop the sequence as
    we keep adding our own predicted characters. Similar to our work in the RNN
    time series problems. '''

    # Number of characters to generate
    num_generate = gen_size

    # Vecotrizing starting seed text
    input_eval = [char_to_ind[s] for s in start_seed]

    # Expand to match batch format shape
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty list to hold resulting generated text
    text_generated = []

    # Temperature effects randomness in our resulting text
    # The term is derived from entropy/thermodynamics.
    # The temperature is used to effect probability of next characters.
    # Higher probability == lesss surprising/ more expected
    # Lower temperature == more surprising / less expected

    temperature = temp

    # Here batch size == 1
    model.reset_states()

    for i in range(num_generate):

      # Generate Predictions
      predictions = model(input_eval)

      # Remove the batch shape dimension
      predictions = tf.squeeze(predictions, 0)

      # Use a cateogircal disitribution to select the next character
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Pass the predicted charracter for the next input
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transform back to character letter
      text_generated.append(ind_to_char[predicted_id])

    return (start_seed + ''.join(text_generated))

In [67]:
print(generate_text(model,"flower",gen_size=1000))

flower;
    Which rand we shall be trusted with their lions,
   her whoough by his friend;
    If I be eving, cross thy mistress, nor I must
    This suit of his particular seal
    When I am gentleman, we should not give.
  PORTIA. Thanks we Padua.
  PARIS. Hark, proceed no question of this curkord.
    Ullf music bark,
    The other England was such princely hairs;
    My valiant and thy heart retires, untrue,
    but she makes both in time, and make him have
    them from the art. 'Tis in love will well enough.
  LAUNCE. Father, I spoke it. I have committed nothing, Montage?
  POLIXENES. They must beat you too much amaz'd me
    A heart usurb'd id you; therefore
   pluce my dreams upon this phelikence,
    Not stol'n interer and the night
    of mine. Hector base is in a barbain queen!
  AARON. Assemen to Thersites, dunge     
  PORTIA. Thou shalt not break the captains, sirs; for Benedick
    Than she, the whimplerise and land spirits was sweet
    As Marcius' tames fool to the 'sc